In [31]:
import string
from copy import copy

from IPython.display import Image, clear_output, display
from Libs.Beholder import *
from PIL import Image as pilImage
from tqdm.notebook import tqdm

In [76]:
a = Beholder(videoFrameGenerator=pullPhoneScreen)

a.addLayerModifer(
    Beholder_Layer_Chopper_Grayscale(name="gray", from_layer="image", enabled=True)
)

a.addLayerModifer(
    Beholder_Layer_Chopper_AtCord(
        name="gray_letters",
        from_layer="gray",
        x=304,
        y=1632,
        w=850,
        h=860,
        enabled=True,
    )
)

In [77]:
from pathlib import Path

import requests

w = Path("words.lst")
if not w.exists():
    w.write_bytes(
        requests.get(
            "https://raw.githubusercontent.com/MariaLuo826/english_words/master/words_alpha.txt"
        ).content
    )
words = [l.upper() for l in w.read_text().splitlines()]
len(words)

370105

In [78]:
a.matchers = {}
for l in string.ascii_uppercase:
    p = Path(f"./wordtrip_templates/{l}.png")
    if p.exists():
        print(f"Loading {l}")
        a.addMatcher(
            Beholder_Image_Matcher(
                name=l,
                layer="gray_letters",
                filename=str(p),
                threshhold=0.9,
                sensitivity=200,
            )
        )

Loading A
Loading B
Loading C
Loading D
Loading E
Loading F
Loading H
Loading I
Loading L
Loading M
Loading N
Loading O
Loading P
Loading R
Loading S
Loading T
Loading U
Loading V
Loading W
Loading Y


In [79]:
def wordWorks(word, found_letters):
    found_letters_t = copy(found_letters)
    for letter in word:
        if letter in found_letters_t:
            found_letters_t.remove(letter)
        else:
            return False
    return True

In [ ]:
min_len = 3
max_len = 5
eq_len = None
matches = a.findMatches()
found_letters = []
for l in matches:
    for _ in range(0, len(matches[l].f)):
        found_letters.append(l)
found_letters

In [ ]:
matching_words = []
for word in tqdm(words):
    if len(word) < min_len or len(word) > max_len:
        continue
    if wordWorks(word, found_letters):
        matching_words.append(word)

plen = 0
for word in list(sorted(matching_words, key=len)):
    if plen != len(word):
        print()
        plen = len(word)
    print(word)